In [61]:
import numpy as np
import pandas as pd
import sympy as sp
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', RuntimeWarning)

In [ ]:
S=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="C:IH").values
F=pd.read_excel(io=r'../data/8-dataset.xlsx',sheet_name='Loss',usecols="B:IG").values
MT=np.diag(1./(pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").values.flatten()))
S=np.dot(MT,S)
supply=np.zeros((402,24,10))
forwa=np.zeros((8,24,10))
for i in range(402):
    supply[i,:,:]=S[i,:].reshape(24,10)
for j in range(8):
    forwa[j,:,:]=F[j,:].reshape(24,10)

In [ ]:
def GM_11(x):
    n=len(x)
    lamda=x[:-1]/x[1:]
    rang=[lamda.min(), lamda.max()]
    theta=[np.exp(-2/(n+1)),np.exp(2/(n+1))]
    x0=x
    if theta[0]<=rang[0] and rang[1]<=theta[1]:
        c=0
    else:
        while True:
            x0=x0+1
            ratio=x0[:-1]/x0[1:]
            if theta[0]<=ratio.min() and ratio.max()<=theta[1]:
                break
        c=x0[0]-x[0]
    x=x+c
    x1=np.cumsum(x)
    z=0.5*(x1[1:]+x1[:-1])
    B=np.vstack([-z,np.ones(n-1)]).T
    u=np.linalg.pinv(B)@x[1:] 
    sp.var('t'); sp.var('xx',cls=sp.Function) 
    eq=xx(t).diff(t)+u[0]*xx(t)-u[1]  
    s=sp.dsolve(eq,ics={xx(0):x0[0]})  
    xt=s.args[1]
    fxt=sp.lambdify(t,xt,'numpy') 
    fo1=fxt(np.arange(n))
    fo=np.hstack([x[0],np.diff(fo1)]) 
    fo=fo-c
    x=x-c
    epsilon=x-fo
    fo=fo[-1]
    return fo,epsilon

In [ ]:
def ARIMA_(x):
    x0=x
    n1=len(x)
    di=0
    while True:
            if adfuller(x0)[1]>=0.5:
                x0=np.diff(x0)
                di=di+1
            else:
                break
    n2=len(x0)
    aicbic=np.empty([0,3])
    for i in range(4):
        for j in range(4):
            if  i==0 and j==0:
                continue
            else:
                model=ARIMA(x0, order=(i,0,j))
                try:
                    model=model.fit()
                    aicbic=np.append(aicbic,[[i,j,model.aic+model.bic]],axis=0)
                except BaseException:
                    aicbic=np.append(aicbic,[[i,j,100]],axis=0)
                    pass
    index=np.where(aicbic==np.min(aicbic[:,2],axis=0))
    mod=ARIMA(x0, order=(aicbic[index[0],0],0,aicbic[index[0],1]))
    fo=mod.fit().forecast(n1+1-n2)
    x_Fo=x[-1]+np.cumsum(fo)
    return x_Fo[-1]

In [ ]:
S_fo=np.zeros((24,402))
F_fo=np.zeros((8,402))
for k in range(402):
    fc=np.zeros(24)
    for l in range(24):
        fo,epsilon=GM_11(supply[k,l,:])
        epsilon=ARIMA(epsilon)
        fc=fo+epsilon[-1]
    S_fo[:,i]=fc
for m in range(8):
    fc=np.zeros(24)
    for n in range(24):
        fo,epsilon=GM_11(supply[m,n,:])
        epsilon=ARIMA(epsilon)
        fc=fo+epsilon[-1]
    F_fo[:,i]=fc

In [63]:
# y1=np.array([1.5539, 1.4642, 2.1018, 1.9554, 2.1018, 2.2294,1.9507, 1.6248, 1.8232, 1.7759])
# y2=np.array([1.639,2.0404,2.645,2.5033,2.3333,2.4372,2.2577,2.201,1.9649,1.6295])
# a,b=GM_11(y1)
# print(a)
# print(b)
# c=ARIMA_(b)
# a+c

1.8661074209135222
[ 0.         -0.45361816  0.19048332  0.05057401  0.20345393  0.33752309
  0.06528152 -0.25417076 -0.04933375 -0.09020742]


2.0163545452597513